# 2016 Election Donations Data Analysis - Data Exploration

## Objectives

- Explore and glean insights from a real dataset using pandas
- Practice using pandas for exploratory analysis, information gathering, and discovery
- Practice cleaning data and answering questions

## Dataset

- Campaign contributions to the 2016 U.S. presidential primary races made in California: https://drive.google.com/file/d/1Lgg-PwXQ6TQLDowd6XyBxZw5g1NGWPjB/view?usp=sharing
- This file originally came from the U.S. Federal Election Commission (https://www.fec.gov/)
- Documentation for this data can be found here: https://drive.google.com/file/d/11o_SByceenv0NgNMstM-dxC1jL7I9fHL/view?usp=sharing

## Data Questions

You are working for a California state-wide election campaign. Your boss wants you to examine historic 2016 election contribution data to see what zipcodes are more supportive of fundraising for your candidate. 

Your boss asks you to filter out some of the records:
- Only use primary 2016 contribution data (more like how your race is).
- Concentrate on Bernie Sanders as a candidate (most a like your candidate)

The questions your boss wants answered is:
- Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
- What day(s) of the month do most people donate?

## Setup

Run the cell below as it will load the data into a pandas dataframe named `contrib`. Note that a custom date parser is defined to speed up loading. If Python were to guess the date format, it would take even longer to load.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Define a date parser to pass to read_csv
d = lambda x: datetime.strptime(x, '%d-%b-%y')

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
contrib = pd.read_csv('../P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)

# Note - for now, it is okay to ignore the warning about mixed types. 

/var/folders/w4/mykv87ns3h3blj3jw28kzwd80000gn/T/ipykernel_92687/3253449235.py:14: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  contrib = pd.read_csv('../P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)
/var/folders/w4/mykv87ns3h3blj3jw28kzwd80000gn/T/ipykernel_92687/3253449235.py:14: DtypeWarning: Columns (6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  contrib = pd.read_csv('../P00000001-CA.csv', index_col=False, parse_dates=['contb_receipt_dt'], date_parser=d)


***
## 1. Initial Data Checks (50 points)

First we will take a preliminary look at the data to check that it was loaded correctly and contains the info we need.

The questions to answer at the end of this section:
- Do we have the correct # of columns and rows. 
- Do the records contain data for the questions we want to answer 
- What columns are important? 
- What columns can be dropped?
- What are the data problems?

**1a.** Print the *shape* of the data. Does this match the expectation? (2 points)

In [6]:
# 1a YOUR CODE HERE
contrib.shape

(1125659, 18)

- **1a answer here:** This shape matches the expectation based on the information provided in the data's documentation. There are 18 columns in the documentation and 18 here and the large number of rows represents all the contributors.

    **My Assumptions in this decision:**
    * I assume the data documentation is correct
    * I assume there are many contributors, which is represented by the large number of rows.

**1b.** Print a list of column names. Are all the columns included that are in the documentation? (2 points)

In [3]:
# 1b YOUR CODE HERE
contrib.columns

Index(['cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city',
       'contbr_st', 'contbr_zip', 'contbr_employer', 'contbr_occupation',
       'contb_receipt_amt', 'contb_receipt_dt', 'receipt_desc', 'memo_cd',
       'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp'],
      dtype='object')

- **1b answer here:** Referencing the documentation, yes, all the columns included in the documentation are in this dataset.

**1c** Print out the first five rows of the dataset. How do the columns `cand_id`, `cand_nm` and `contbr_st` look? (3 points)

In [4]:
# 1c YOUR CODE HERE
contrib.head()

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00575795,P00003392,"Clinton, Hillary Rodham","AULL, ANNE",LARKSPUR,CA,"949,391,913.00",NaN,RETIRED,50.00,2016-04-26,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4768722,P2016
1,C00575795,P00003392,"Clinton, Hillary Rodham","CARROLL, MARYJEAN",CAMBRIA,CA,"934,284,638.00",NaN,RETIRED,200.00,2016-04-20,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4747242,P2016
2,C00575795,P00003392,"Clinton, Hillary Rodham","GANDARA, DESIREE",FONTANA,CA,"923,371,507.00",NaN,RETIRED,5.00,2016-04-02,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4666603,P2016
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,"930,111,214.00",AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016


- **1c answer here:** cand_id is in line with the data documentation "Candidate ID- a String of a 9-character alpha-numeric code assigned to a candidate by the Federal Election Commission." Each value in these 5 rows are 9 characters long and the cand_id is specific to the cand_nm (Candidate's name as described in the data documentation.) The cand_nm also looks like what we would expect. It is the candidate's full name in this order: [last name/surname], [first name]. If the candidate has two last names, then it is displayed in this order: [second last name/surname], [first name] [first last name/surname]. 

    The contbr_st is also in line with what's in the documentation "Contributer State- a String of the reported state of the contributor." However, it is the abbreviated state name, ex: California >> CA.

    **My Assumptions:**
    - The cand_id and cand_nm value pairs should remain consistent.


**1d.** Print out the values for the column `election_tp`. In your own words, based on the documentation, what information does the `election_tp` variable contain? Do the values in the column match the documentation? (3 points)

In [5]:
# 1d YOUR CODE HERE
print("The unique values in this column:", contrib.election_tp.unique())
contrib.election_tp

The unique values in this column: ['P2016' 'G2016' nan 'P2020']


0          P2016
1          P2016
2          P2016
3          P2016
4          P2016
           ...  
1125654    P2016
1125655    P2016
1125656    P2016
1125657    P2016
1125658    P2016
Name: election_tp, Length: 1125659, dtype: object

- **1d answer here:** The information in this variable represents the election type or primary-general indicator and the election year as a String. The first character in a value is a P if it's a primary election or a G if it's a general election and the following 4 characters are the election year. 

    The information matches the documentation in terms of the primary/general code (P/G) and the format election, election year (EYYYY). However, the unique values show there are only Primary and General election types in this data and doesn't list any of the other 5 election types.


**1e.** Print out the datatypes for all of the columns. What are the datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt`? (5 points)

In [6]:
# 1e YOUR CODE HERE
print("Example value and Type for contbr_zip:", contrib.contbr_zip[0],type(contrib.contbr_zip[0]))
print("Example value and Type for contb_receipt_amt:", contrib.contb_receipt_amt[0], type(contrib.contb_receipt_amt[0]))
print("Example value and Type for contb_receipt_dt:",contrib.contb_receipt_dt[0], type(contrib.contb_receipt_dt[0]))
contrib.dtypes

Example value and Type for contbr_zip: 949391913.0 <class 'float'>
Example value and Type for contb_receipt_amt: 50.0 <class 'numpy.float64'>
Example value and Type for contb_receipt_dt: 2016-04-26 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


cmte_id                      object
cand_id                      object
cand_nm                      object
contbr_nm                    object
contbr_city                  object
contbr_st                    object
contbr_zip                   object
contbr_employer              object
contbr_occupation            object
contb_receipt_amt           float64
contb_receipt_dt     datetime64[ns]
receipt_desc                 object
memo_cd                      object
memo_text                    object
form_tp                      object
file_num                      int64
tran_id                      object
election_tp                  object
dtype: object

- **1e:** The datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt` variables based on the dataset are:
    - `contbr_zip`: object, individual items are float type
        - data documentation says this type is string. Contradicts what dataset shows
        - format is 9 digit but 5 digit may be better
        - there are different formats (commas, no commas)
    - `contb_receipt_amt`: float64
        -  data documentation says this type is numeric. In line with this dataset.
    - `contb_receipt_dt`: datetime64[ns], individual items are timestamp
        - data documentation says this type is date. In line with this dataset.


**1f.** What columns have the most non-nulls?  Would you recommend to drop any columns based on the number of nulls? (5 points)

In [7]:
# 1e YOUR CODE HERE
contrib.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125659 entries, 0 to 1125658
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   cmte_id            1125659 non-null  object        
 1   cand_id            1125659 non-null  object        
 2   cand_nm            1125659 non-null  object        
 3   contbr_nm          1125659 non-null  object        
 4   contbr_city        1125633 non-null  object        
 5   contbr_st          1125659 non-null  object        
 6   contbr_zip         1125564 non-null  object        
 7   contbr_employer    967757 non-null   object        
 8   contbr_occupation  1115260 non-null  object        
 9   contb_receipt_amt  1125659 non-null  float64       
 10  contb_receipt_dt   1125659 non-null  datetime64[ns]
 11  receipt_desc       15045 non-null    object        
 12  memo_cd            144268 non-null   object        
 13  memo_text          501148 n

- **1f answer here:** The columns with the most non-nulls are cmte_id, cand_id, cand_nm, contbr_nm, contbr_st, contb_receipt_amt, contb_receipt_dt, form_tp, file_num, and tran_id. They all have the "1125659" values which is equivalent to the total number of entries, meaning there is no missing data for any of these.

    To decide whether or not to drop any columns, we should see what columns are/aren't relevant to our research question. I would not recommend to drop columns based only on the number of nulls since it may be sufficient to drop the rows where there are null values. We should first determine if the column is relevant to our questions.

    In our case, we specifically want data on primary 2016 elections and candidate Bernie Sanders to deliver information on zipcodes based on corresponding contributions and dollar amount. We also want to determine the days of the month most people donate. For example, even though `election_tp` has less non-nulls/more nulls, we will need this information to filter our data so we should not drop it. There are a few columns we can drop because they are not relevant to our analysis, like receipt description(receipt_desc), memo code (memo_cd) and memo text (memo_text) which all have more null values and there are also some that have no null values but are irrelevant so we can also drop them, like form type (form_tp).

    **My Assumptions:**
    - At this stage, I assume that missing data is only represented by null/nan values, not 0 or Boolean values.

**1g.** A column we know that we want to use is the cand_nm column.  From the documentation each candidate is a unique candidate id also. Check data quality of `cand_id` column to see if it matches `cand_nm` column. Specifically check to ensure our targetted candidate 'Bernard Sanders' always has the same cand_id throughout. Any issues with `cand_nm` matching `cand_id`? (5 points)

In [8]:
match_cand = contrib[["cand_id", "cand_nm"]]
print(f"There are {len(match_cand['cand_id'].unique())} unique candidate id's (cand_id).")
print(f"There are {len(match_cand['cand_nm'].unique())} unique candidate names (cand_nm).")

There are 25 unique candidate id's (cand_id).
There are 25 unique candidate names (cand_nm).


In [9]:
# Look to see if each candidate id corresponds to only one candidate name
print(f'There are {len(match_cand.drop_duplicates())} unique cand_id, cand_nm pairs.')
match_cand.drop_duplicates()

There are 25 unique cand_id, cand_nm pairs.


,cand_id,cand_nm
0,P00003392,"Clinton, Hillary Rodham"
3,P60007168,"Sanders, Bernard"
10,P60006111,"Cruz, Rafael Edward 'Ted'"
8884,P60006046,"Walker, Scott"
8899,P60008059,"Bush, Jeb"
9745,P60006723,"Rubio, Marco"
10957,P60003670,"Kasich, John R."
11528,P60008521,"Christie, Christopher J."
14682,P20002671,"Johnson, Gary"
15680,P80001571,"Trump, Donald J."


- **1g answer here:** Based on these results, there don't seem to be any issues with cand_id and cand__nm always matching. There are 25 unique cand_id, 25 unique cand_nm, and 25 unique cand_id, cand_nm pairs. In this last table we see that Bernie Sanders is assigned only one unique id. 
    - cand_id: P60007168
    - cand_nm: Sanders, Bernard

**My assumptions:**
- All candidate id and candidate name pairs are unique and the id's are unique to only one name.

**1h.** Another area to check is to make sure all of the records are from California. Check the `contbr_st` column - are there any records outside of California based on `contbr_st`? (5 points)

In [10]:
contrib["contbr_st"].unique()

array(['CA'], dtype=object)

- **1h answer here:** From the contbr_st column alone, there does not seem to be any records outside of California

**My Assumptions**
- This dataset only has data from California contributors.

**1i.** The next column to check for the analysis is the `tran_id` column. This column could be the primary key so look for duplicates. How many duplicate entries are there? Any pattern for why are there duplicate entries? (5 points)

In [11]:
contrib["tran_id"].describe()

count                  1125659
unique                 1122205
top       ADB49CB248C174E298F0
freq                         4
Name: tran_id, dtype: object

In [12]:
dup_id = contrib[contrib["tran_id"].duplicated(keep=False)]
print(f"There are {len(dup_id['tran_id'])} duplicates for the tran_id variable.")
print(f'There are {len(dup_id["tran_id"].unique())} unique ids among tran_id duplicates.')

There are 6873 duplicates for the tran_id variable.
There are 3419 unique ids among tran_id duplicates.


In [13]:
# To do analysis, I will only look at columns that are important for our questions
# In our case, we specifically want data on primary 2016 elections and candidate Bernie Sanders 
# to deliver information on zipcodes based on corresponding contributions and dollar amount. 
# We also want to determine the days of the month most people donate.
dup_id = dup_id[["cand_id", "cand_nm", "contbr_zip","contb_receipt_amt","contb_receipt_dt","tran_id", "election_tp"]]
dup_id.head()

,cand_id,cand_nm,contbr_zip,contb_receipt_amt,contb_receipt_dt,tran_id,election_tp
57,P00003392,"Clinton, Hillary Rodham","917,733,736.00",151.00,2016-03-31,C3913687,P2016
90,P00003392,"Clinton, Hillary Rodham","945,365,810.00",300.00,2016-03-31,C3925472,P2016
91,P00003392,"Clinton, Hillary Rodham","919,775,125.00",100.00,2016-03-31,C3937968,P2016
174,P60006111,"Cruz, Rafael Edward 'Ted'","930,229,619.00",35.00,2016-04-12,SA17A.1645838,P2016
175,P60006111,"Cruz, Rafael Edward 'Ted'","928,065,615.00",5.00,2016-04-06,SA17A.1627230,P2016


In [14]:
for col in dup_id.columns:
    print(f'column name:  {col}')
    print(f'number of unique values: {len(dup_id[col].unique())}')
    print(dup_id[col].value_counts(dropna=False))
    print("-" * 50)

column name:  cand_id
number of unique values: 15
cand_id
P00003392    2102
P60006111    2015
P60005915    1378
P60006723     863
P60007242     155
P60008059      87
P80001571      84
P20003984      59
P20002671      43
P60007697      35
P40003576      25
P80003478      15
P60006046       5
P60008521       5
P60022654       2
Name: count, dtype: int64
--------------------------------------------------
column name:  cand_nm
number of unique values: 15
cand_nm
Clinton, Hillary Rodham      2102
Cruz, Rafael Edward 'Ted'    2015
Carson, Benjamin S.          1378
Rubio, Marco                  863
Fiorina, Carly                155
Bush, Jeb                      87
Trump, Donald J.               84
Stein, Jill                    59
Johnson, Gary                  43
Graham, Lindsey O.             35
Paul, Rand                     25
Huckabee, Mike                 15
Walker, Scott                   5
Christie, Christopher J.        5
McMullin, Evan                  2
Name: count, dtype: int64
-

In [15]:
contrib["cand_id"].value_counts()
print(f'Bernie Sanders appears {len(contrib[contrib["cand_id"] == "P60007168"].value_counts())} times in the contrib dataset.')
print(f'and {len(dup_id[dup_id["cand_id"] == "P60007168"].value_counts())} in the duplicate transaction ids dataset.')

Bernie Sanders appears 28 times in the contrib dataset.
and 0 in the duplicate transaction ids dataset.


In [16]:
print('The most frequent tran_ids are "ADB49CB248C174E298F0" and "A5602AD777C8C4632B5A" both appearing 4 times')
dup_id[dup_id.tran_id.isin(["A5602AD777C8C4632B5A", "ADB49CB248C174E298F0"])]

The most frequent tran_ids are "ADB49CB248C174E298F0" and "A5602AD777C8C4632B5A" both appearing 4 times


,cand_id,cand_nm,contbr_zip,contb_receipt_amt,contb_receipt_dt,tran_id,election_tp
40217,P40003576,"Paul, Rand","92,075.00",500.00,2015-09-30,ADB49CB248C174E298F0,P2016
40218,P40003576,"Paul, Rand","953,260,412.00",200.00,2015-09-30,A5602AD777C8C4632B5A,G2016
41393,P40003576,"Paul, Rand","953,260,412.00",200.00,2015-09-30,A5602AD777C8C4632B5A,G2016
42430,P40003576,"Paul, Rand","92,075.00",500.00,2015-09-30,ADB49CB248C174E298F0,P2016
42822,P40003576,"Paul, Rand","92,075.00",500.00,2015-09-30,ADB49CB248C174E298F0,P2016
42836,P40003576,"Paul, Rand","953,260,412.00",200.00,2015-09-30,A5602AD777C8C4632B5A,G2016
44681,P40003576,"Paul, Rand","92,075.00",500.00,2015-09-30,ADB49CB248C174E298F0,P2016
44685,P40003576,"Paul, Rand","953,260,412.00",200.00,2015-09-30,A5602AD777C8C4632B5A,G2016


- **1i answer here:** There are 6873 duplicate tran_id values out of the 1125659 entries. 

    **Patterns:**
    
    Description: The tran_id represents transaction id which is a unique identifir that's permanently associated with each itemization or transaction. 

    Background: In our case, we specifically want data on primary 2016 elections and candidate Bernie Sanders to deliver information on zipcodes based on corresponding contributions and dollar amount. We also want to determine the days of the month most people donate. His cand_id: P60007168 and cand_nm: Sanders, Bernard

    Our candidate, Bernie Sanders appears 28 times in the contrib dataset but does not appear in the duplicate dataset. These are the most frequently appearing values in the duplicate dataset for our variables of interest based on our question:
    - tran_ids: "ADB49CB248C174E298F0" and "A5602AD777C8C4632B5A", count: 4 times each
    - candidate: Clinton, Hillary Rodham,  count:2102
    - contributor zip code: 953554609, count: 31
    - dollar amount contribution: 100.00, count: 1228
    - contribute date: 2016-06-30, count: 216
    - election type and year: P2016, count: 6144

**1j.** Another column to check is the `contb_receipt_amt` that shows the donation amounts. How many negative donations are included? What do negative donations mean? Please show at least pull a few rows to look at the records with negative donations. Do these records match with the expectation of why a negative donation would happen? (5 points)

In [17]:
print(f'There are {len(contrib[contrib["contb_receipt_amt"] < 0]["contb_receipt_amt"])} negative donations')
neg_donations = contrib[contrib["contb_receipt_amt"] < 0]
neg_donations.head()

There are 11896 negative donations


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
19,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-25.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826482B,P2016
23,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-150.00,2016-04-29,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826483B,P2016
81,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","JOLLIFF, RICHARD",CHICO,CA,"959,289,507.00",SELF EMPLOYED,RANCHER,-60.00,2016-04-14,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1827494,P2016
190,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","NOWELL, DIANA L.",RANCHO SANTA MARGARITA,CA,"926,884,928.00",CAPISTRAND UNIFIED SCHOOL DISTRICT,LIBRARIAN TECHNICIAN,-100.00,2016-04-11,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1639830B,P2016
213,C00574624,P60006111,"Cruz, Rafael Edward 'Ted'","LICHTY, ANDREW MR.",SAN DIEGO,CA,"921,096,720.00",SELF EMPLOYED,REAL ESTATE,-25.00,2016-04-30,REDESIGNATION TO GENERAL,X,REDESIGNATION TO GENERAL,SA17A,1077664,SA17A.1826888B,P2016


In [18]:
print(list(neg_donations["receipt_desc"].unique()))

['REDESIGNATION TO GENERAL', 'Refund', nan, 'REATTRIBUTION TO SPOUSE', 'REDESIGNATION TO PRESIDENTIAL GENERAL', '2016 SENATE PRIMARY DONOR REDESIGNATION TO GENERAL', 'REDESIGNATION FROM PRIMARY', 'US CITIZEN REDESIGNATION TO GENERAL', 'REDESIGNATION TO CRUZ FOR SENATE', '* REATTRIBUTED TO JOHN CASTOR', '* REATTRIBUTED TO FRANK DARABONT', 'REFUND TO BE ISSUED; REDESIGNATION TO GENERAL', 'REATTRIBUTION TO SPOUSE; SEE REDESIGNATION', '* REATTRIBUTED TO CHRISTINE MARSHALL', '* REATTRIBUTED TO CAROL FITZMORRIS', '* REATTRIBUTED TO ERIC UHRANE', '* REATTRIBUTED TO YEU-WEI YEE', '* REATTRIBUTED TO NORAH STONE', '* REATTRIBUTED FROM KIM RIDDER', 'REDESIGNATION TO PRIMARY', 'REATTRIBUTION TO SPOUSE, TERRIE SCHULTZ.', '* REATTRIBUTION TO SPOUSE, TERRIE SCHULTZ.', 'REATTRIBUTION TO SPOUSE, HARRY HICKS.', 'REATTRIBUTION/REDESIGNATION REQUESTED REDESIGNATION TO GENERAL', '* REATTRIBUTED TO ELIZABETH DOYLE', '* REATTRIBUTED TO PRUDENCE MOORE', '* REATTRIBUTED TO AISHA LYNCH', '* REATTRIBUTED TO SHEI

- **1j answer here:** There are 11896 negative donations.
    Based on the documentation, I expect the negative amount to represent the refunds to individuals and/or transfers from authorized committees. 

    Based on this website info: (https://www.fec.gov/help-candidates-and-committees/taking-receipts-political-party/refunds-contributions/)
    In the case that a committee receives a refund of a contribution it has made, it must report it and follow certain protocols.
    In particular, **if an original check to a candidate committee or other political committee is not deposited, such as it's returned,** 
    **uncashed or lost, the refund amount must be reported as a negative entry**. Therefore, a negative donation means that the original
    check was not deposited.

    Additionally, based on this site: https://www.fec.gov/help-candidates-and-committees/filing-reports/redesignating-and-reattributing-contributions/ 
    "When a committee receives an excessive contribution—one which exceeds the contributor’s limit or the campaign’s net debts outstanding for an election—the committee may remedy the violation by **refunding the excessive amount or by seeking a redesignation or reattribution** of it within 60 days."

    These records match with the expectation of why a negative donation would happen since the receipt_desc column which has additional 
    information reported by the committee shows these 3 explainations: refunds, redisignation, and reattribution


**1k.** One more column to look at is the date of donation column. Are there any dates outside of the primary period (defined as 1 Jan 2014 to 7 June 2016)? Are the dates well-formatted for our analysis? (5 points)

In [19]:
# The date format is DD-MMM-YYYY.
nonprimary_dates = contrib[(contrib["contb_receipt_dt"] < "2014-01-01") | (contrib["contb_receipt_dt"] > "2016-06-07")]["contb_receipt_dt"]
print(f'The type for these values are {type(list(nonprimary_dates)[0])}.')
nonprimary_dates

The type for these values are <class 'pandas._libs.tslibs.timestamps.Timestamp'>.


9932      2013-11-05
9994      2013-11-05
14673     2016-06-30
14682     2016-07-20
14697     2016-07-14
             ...    
1123992   2016-08-02
1123993   2016-08-09
1123994   2016-08-31
1123995   2016-08-31
1123996   2016-07-06
Name: contb_receipt_dt, Length: 451372, dtype: datetime64[ns]

- **1k answer here:** I used the contb_receipt_dt column to see if any dates fall outside our primary election period since it's the contribution receipt date
    and we want the date of donation. 
    [(contrib["contb_receipt_dt"] < "2014-01-01") | (contrib["contb_receipt_dt"] > "2016-06-07")]
    There are 451372 dates outside of our primary period. 2 dates fall before the primary period and the rest fall after the primary.The type for these values
    is a pandas timestamp which is easy to use for our analysis, but the date format is unconventional for U.S date format and is inconsistent with the given primary
    date definition (DD-MM-YYYY) and what is actually seen and used in the dataset (YYYY-MM-DD). I think the more intuitive format would be MM-DD-YYYY using US formatting. 
    However this format is still manageable.

**1l.** Finally, answer the initial questions in the cells below (5 points)

**1l.1** Do we have the correct # of columns and rows.

- **1l.1 answer here:** Yes, this shape (1125659, 18) matches the expectation based on the information provided in the data's documentation. There are 18 columns in the documentation and 18 here and the large number of rows represents all the contributors.

**1l.2** Do the records contain data for the questions we want to answer?

- **1l.2 answer here:**  Yes, these records contain the information we need to answer our questions. Our questions are: 
    - Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
    - What day(s) of the month do most people donate?

    These records contain data on the total number of contributions, refunds, and transfers and contains data on the primary 2016 elections, our candidate Bernie Sanders, the zipcodes based on corresponding contributions, contribution dollar amount and the contribution receipt date.

**1l.3** What columns are important?

- **1l.3 answer here:** Based on the questions we want to answer and the steps we need to get there, the following columns are important:
    - `cand_id`: string - all entries
    - `cand_nm`: string - all entries
    - `contbr_zip`: object - 1125564 non-null entries out of 1125659 total entries
    - `contb_receipt_amt`: float64 - all entries
    - `contb_receipt_dt`: datetime64[ns] - all entries
    - `memo_text`: string - 501148 non-null entries out of 1125659 total entries - may be helpful if we have questions on the contribution
    - `tran_id`: string - all entries
    - `election_tp`: string - 1124234 non-null out of 1125659 total entries

**1l.4** What columns can be dropped?

- **1l.4 answer here:** Based on the questions we want to answer and the steps we need to get there, the following columns are not relevant or don't contribute
to our analysis, so they can be dropped:
    - `cmte_id`, `contbr_nm`, `contbr_city`, `contbr_st`, `contbr_employer`, `contbr_occupation`, `memo_cd`,`receipt_desc`, `form_tp`, `file_num`

**1l.5** What are the data problems?

- **1l.5 answer here:** 
    **Data Type**
    - The type for the values in the `cont_receipt_dt` variable are pandas timestamp which is easy to use for our analysis, but the date format is unconventional for U.S date format and is inconsistent with the given primary date definition (DD-MM-YYYY) and what is actually seen and used in the dataset (YYYY-MM-DD).
        - I think the more intuitive format would be MM-DD-YYYY using US formatting. However this format is still manageable.
    - There seems to be discrepencies between what the data types are in this dataset and what is displayed in the data documentation.
    Specifically for `contbr_zip`. The data shows that its dtype is object and type float for individual items but the data documentation
    says this type is string. 
        - This format is 9 digits, so the conventional 5 digits may be better to use.
        - There are different formats (commas, no commas) in the zipcodes, so we should strip it.

    **Duplicates and Null**
    - There are 6873 duplicates for the tran_id variable and 3419 unique ids among the tran_id duplicates. 
        - This may not be relevant since we're focusing on Bernie Sanders and there are no duplicate tran_ids for his cand_id
    - There are null values in the `contbr_zip` and `election_tp` columns which may create issues since these variable are essential for our questions
    and instructions to filter dataset.
        - may need to drop null rows in these columns
        - may be able to use only the `contb_receipt_ft` variable to determine 2016 primary election contributions which may be better since
        it has all the entries
        - `memo_text` is also missing values but this is not essential to our questions so it may be ok to keep null rows if it doesn't interfere
        with out analysis
    

**1l.6** List any assumptions so far:

- **1l.6 answer here:** 
    - I assume the data documentation is correct
    - I assume there are many contributors, which is represented by the large number of rows.
    - The cand_id and cand_nm value pairs should remain consistent.
    - At this stage, I assume that missing data is only represented by null/nan values, not 0 or Boolean values.
    - All candidate id and candidate name pairs are unique and the id's are unique to only one name.
    - This dataset only has data from California contributors.
    - Negative contribution amounts are because of refunds received by a committee.
    - We have sufficient data that falls within our primary period to do our analysis.

***
## 2. Data filtering and data quality fixes (30 points)

Now that we have a basic understanding of the data, let's filter out the records we don't need and fix the data.

**2a.** From the dataset filter out (remove) any election_tp not in the primary election. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [20]:
# The date format is YYYY-MM-DD and the primary period is defined as 1 Jan 2014 to 7 June 2016
primary_contrib = contrib[(contrib["contb_receipt_dt"] >= "2014-01-01") & (contrib["contb_receipt_dt"] <= "2016-06-07")]
print("Shape of the dataframe after filtering for only primary election contribution dates:", primary_contrib.shape)
primary_contrib.head(3)

Shape of the dataframe after filtering for only primary election contribution dates: (674287, 18)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00575795,P00003392,"Clinton, Hillary Rodham","AULL, ANNE",LARKSPUR,CA,"949,391,913.00",NaN,RETIRED,50.00,2016-04-26,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4768722,P2016
1,C00575795,P00003392,"Clinton, Hillary Rodham","CARROLL, MARYJEAN",CAMBRIA,CA,"934,284,638.00",NaN,RETIRED,200.00,2016-04-20,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4747242,P2016
2,C00575795,P00003392,"Clinton, Hillary Rodham","GANDARA, DESIREE",FONTANA,CA,"923,371,507.00",NaN,RETIRED,5.00,2016-04-02,NaN,X,* HILLARY VICTORY FUND,SA18,1091718,C4666603,P2016


**2b.** From the dataset filter out (remove) any candidate that is not Bernie Sanders. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [21]:
# We filter for only Bernie Sanders contributions by using his unique cand_id: P60007168 
primary_contrib = primary_contrib[primary_contrib["cand_id"] == "P60007168"]
print("Shape of the dataframe after filtering to only keep Bernie Sanders:", primary_contrib.shape)
primary_contrib.head()

Shape of the dataframe after filtering to only keep Bernie Sanders: (379285, 18)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,"930,111,214.00",AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016
5,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,"902,784,310.00",VERICOR ENTERPRISES INC.,PHARMACIST,100.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYBXV4,P2016
6,C00577130,P60007168,"Sanders, Bernard","LEOPARD, PATTI",VISTA,CA,"920,842,849.00",ONSITE ENERGY CORPORATION,PROJECT MANAGER,25.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKW04C1,P2016
8,C00577130,P60007168,"Sanders, Bernard","LEPKE, KELLY",WESTMINSTER,CA,"926,833,846.00",NONE,NOT EMPLOYED,10.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3H59,P2016


**2c.** The `contbr_zip` column is not formatted well for our analysis. Make a new zipcode column that is the five-digit zipcodes. Filter out any records outside of California based on the zipcode. Print/show the shape of the dataframe after the filtering is complete. (10 points).

- You will have to research what the valid 5-digit zipcodes for California are!

In [22]:
# We need to strip all punctuation from current zip codes and condense
# them to only 5 digits
new_zips = []
for zip in primary_contrib["contbr_zip"]:
    zip = str(zip).strip()
    zip = zip[:5]
    new_zips.append(zip)
primary_contrib["contbr_zip"] = new_zips
primary_contrib.head(3)

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
3,C00577130,P60007168,"Sanders, Bernard","LEE, ALAN",CAMARILLO,CA,93011,AT&T GOVERNMENT SOLUTIONS,SOFTWARE ENGINEER,40.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKWA097,P2016
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016
5,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,100.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYBXV4,P2016


In [23]:
# Need to select only CA zips so will import dataset to filter for these
# dataset from: https://gis.data.ca.gov/datasets/CDEGIS::california-zip-codes/about
ca_zip = pd.read_csv('../ZipCodes.csv', index_col=False)
print("Shape of this CA zipcodes dataset:", ca_zip.shape)
print("Ex Value and Type for zip code values:",ca_zip["Zip Code"][0], type(ca_zip["Zip Code"][0]))
ca_zip.head()

Shape of this CA zipcodes dataset: (1721, 9)
Ex Value and Type for zip code values: 12 <class 'numpy.int64'>


,OBJECTID,Zip Code,Post Office Name,State,Pop 2017,Pop SqMile 2017,SqMile,Shape__Area,Shape__Length
0,1,12,Mt Meadows Area,CA,-99,-99.00,30.92,"862,157,443.22","195,388.62"
1,2,16,Sequoia National Forest,CA,-99,-99.00,39.33,"1,096,295,677.53","169,790.57"
2,3,17,Northeast Fresno County,CA,-99,-99.00,564.38,"15,734,145,627.65","873,109.21"
3,4,18,Los Padres Ntl Forest,CA,-99,-99.00,90.83,"2,532,221,635.86","294,311.33"
4,5,19,Lassen Ntl Forest,CA,-99,-99.00,412.59,"11,502,609,569.71","533,289.13"


In [24]:
# To filter CA Zip Codes only, first need to match data types for zip code variables
new = []
for val in ca_zip["Zip Code"]:
    val = str(val)
    new.append(val)
ca_zip["Zip Code"] = new

# Filter our dataset for only CA zip codes
primary_contrib = primary_contrib[primary_contrib["contbr_zip"].isin(ca_zip["Zip Code"])]
print("Shape of the dataframe after filtering to only keep CA Zip Codes:", primary_contrib.shape)
print("Difference in rows from original primary_contrib dataframe with shape (379285, 18):", 379285 - primary_contrib.shape[0])
primary_contrib.head()

Shape of the dataframe after filtering to only keep CA Zip Codes: (370393, 18)
Difference in rows from original primary_contrib dataframe with shape (379285, 18): 8892


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016
5,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,100.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYBXV4,P2016
6,C00577130,P60007168,"Sanders, Bernard","LEOPARD, PATTI",VISTA,CA,92084,ONSITE ENERGY CORPORATION,PROJECT MANAGER,25.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKW04C1,P2016
8,C00577130,P60007168,"Sanders, Bernard","LEPKE, KELLY",WESTMINSTER,CA,92683,NONE,NOT EMPLOYED,10.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3H59,P2016
9,C00577130,P60007168,"Sanders, Bernard","LERCH, DOUGLAS",PETALUMA,CA,94952,SEEDS OF AWARENESS,DIRECTOR OF NON PROFIT,15.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYY720,P2016


**2d.** The receipt amount column has negative donations. After talking with your team, a decision was made that the best course of action is to remove these negative values so that the donation count and amount is more accurate. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [25]:
# 2d YOUR CODE HERE
primary_contrib = primary_contrib[primary_contrib["contb_receipt_amt"] >= 0]
print("Shape of the dataframe after filtering to remove negative donation values:", primary_contrib.shape)
print("Difference in rows from previos primary_contrib dataframe with shape (370393, 18):", 370393 - primary_contrib.shape[0])
primary_contrib.head()

Shape of the dataframe after filtering to remove negative donation values: (368134, 18)
Difference in rows from previos primary_contrib dataframe with shape (370393, 18): 2259


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
4,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,35.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3MB3,P2016
5,C00577130,P60007168,"Sanders, Bernard","LEONELLI, ODETTE",REDONDO BEACH,CA,90278,VERICOR ENTERPRISES INC.,PHARMACIST,100.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYBXV4,P2016
6,C00577130,P60007168,"Sanders, Bernard","LEOPARD, PATTI",VISTA,CA,92084,ONSITE ENERGY CORPORATION,PROJECT MANAGER,25.00,2016-03-04,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKW04C1,P2016
8,C00577130,P60007168,"Sanders, Bernard","LEPKE, KELLY",WESTMINSTER,CA,92683,NONE,NOT EMPLOYED,10.00,2016-03-05,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKX3H59,P2016
9,C00577130,P60007168,"Sanders, Bernard","LERCH, DOUGLAS",PETALUMA,CA,94952,SEEDS OF AWARENESS,DIRECTOR OF NON PROFIT,15.00,2016-03-06,NaN,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,SA17A,1077404,VPF7BKYY720,P2016


**2e.** From the dataset drop any columns that won't be used in the analysis. Print/show the shape of the dataframe after the dropping is complete. What columns did you drop and why? (5 points)

In [26]:
primary_contrib = primary_contrib.drop(columns=['cmte_id', 'contbr_nm', 'contbr_city', 'contbr_st',\
                                                 'contbr_employer', 'contbr_occupation', 'memo_cd',\
                                                    'receipt_desc', 'form_tp', 'file_num'])

In [27]:
print("Shape of the dataframe after dropping irrelevant columns:", primary_contrib.shape)
print("Difference in columns from previos primary_contrib dataframe with shape (368134, 18):", 18 -6)
primary_contrib.head()

Shape of the dataframe after dropping irrelevant columns: (368134, 8)
Difference in columns from previos primary_contrib dataframe with shape (368134, 18): 12


,cand_id,cand_nm,contbr_zip,contb_receipt_amt,contb_receipt_dt,memo_text,tran_id,election_tp
4,P60007168,"Sanders, Bernard",90278,35.00,2016-03-05,* EARMARKED CONTRIBUTION: SEE BELOW,VPF7BKX3MB3,P2016
5,P60007168,"Sanders, Bernard",90278,100.00,2016-03-06,* EARMARKED CONTRIBUTION: SEE BELOW,VPF7BKYBXV4,P2016
6,P60007168,"Sanders, Bernard",92084,25.00,2016-03-04,* EARMARKED CONTRIBUTION: SEE BELOW,VPF7BKW04C1,P2016
8,P60007168,"Sanders, Bernard",92683,10.00,2016-03-05,* EARMARKED CONTRIBUTION: SEE BELOW,VPF7BKX3H59,P2016
9,P60007168,"Sanders, Bernard",94952,15.00,2016-03-06,* EARMARKED CONTRIBUTION: SEE BELOW,VPF7BKYY720,P2016


- **2e answer here:** Based on the questions we want to answer and the steps we need to get there, the following
columns are not relevant or don't contribute to our analysis, so they can be dropped:
`cmte_id`, `contbr_nm`, `contbr_city`, `contbr_st`, `contbr_employer`, `contbr_occupation`,
`memo_cd`,`receipt_desc`, `form_tp`, `file_num`

**2f.** List any assumptions that you made up to this point:

- **2f answer here:** 
    - I assume the data documentation is correct
    - I assume there are many contributors, which is represented by the large number of rows.
    - The cand_id and cand_nm value pairs should remain consistent.
    - I assume that missing data is only represented by null/nan values, not 0 or Boolean values.
    - All candidate id and candidate name pairs are unique and the id's are unique to only one name.
    - This dataset only has data from California contributors.
    - Negative contribution amounts are because of refunds received by a committee.
    - We have sufficient data that falls within our primary period to do our analysis.
    - The zipcodes in this dataset all have the format of 5 valid digits + 4 trailing digits,
    so I could just slice for the first 5 digits
    - We would like to include 0 values (if any) for contribution amounts

***
## 3. Answering the questions (20 points)

Now that the data is cleaned and filterd - let's answer the two questions from your boss!

**3a.** Which zipcode had the highest count of contributions and the most dollar amount? (10 points)

In [28]:
# Looking at current data to get an understanding of what we're looking for.

# Identify the zipcode with the highest count of contributions by identifying the
# most frequently occuring zipcode, shown by top and freq below
primary_contrib["contbr_zip"].describe()

count     368134
unique      1453
top        94110
freq        3799
Name: contbr_zip, dtype: object

In [29]:
# only look at the columns of interest ["contbr_zip","contb_receipt_amt"] and sort them to
# get an idea of the max zip code and contribution amount
temp = primary_contrib.sort_values(by=["contbr_zip","contb_receipt_amt"], ascending = False)[["contbr_zip","contb_receipt_amt"]]
temp

,contbr_zip,contb_receipt_amt
153990,96162,250.00
184073,96162,83.45
4416,96162,50.00
47575,96162,50.00
47977,96162,50.00
...,...,...
82523,90003,4.00
82516,90003,3.00
444279,90002,500.00
240694,90002,35.00


In [30]:
# Create a new dataframe for our analysis. 
# Groupby the zip code and obtain the count of each zip code/frequency
max_zip_amt = primary_contrib[["contbr_zip","contb_receipt_amt"]].groupby(by = ["contbr_zip"]).count().rename(columns = {"contb_receipt_amt": "contb_counts"})

# Groupby the zip code and sum the contribution amounts for the each zip code
max_zip_amt["contb_total_amt"] = primary_contrib[["contbr_zip","contb_receipt_amt"]].groupby(by = ["contbr_zip"]).sum()["contb_receipt_amt"]

# Sort the table, first by zip code counts then contribution amount totals to get max values
max_zip_amt.sort_values(by = ["contb_counts","contb_total_amt"], ascending=False)

,contb_counts,contb_total_amt
contbr_zip,,
94110,3799,"284,398.05"
94114,2857,"201,845.28"
94117,2729,"195,159.39"
95060,2660,"140,018.74"
94611,2340,"144,777.25"
...,...,...
95987,1,25.00
91330,1,15.00
95701,1,15.00


- **3a answer here:**  The zipcode with the highest count of contributions and 
    the most dollar amount is 94110. It had 3799 contributions and a contribution
    amount total of $284,398.05.

**3b.** What day(s) of the month do most people donate? (10 points)

In [31]:
# Look at the counts for each date in the primary election dataset
unique_dates = primary_contrib["contb_receipt_dt"].value_counts().reset_index()
print(f'There are {len(unique_dates)} unique dates in this dataset.')
unique_dates.head()

There are 388 unique dates in this dataset.


,contb_receipt_dt,count
0,2016-02-29,8302
1,2016-03-31,8244
2,2016-05-31,6872
3,2016-04-30,6479
4,2016-03-09,6356


In [32]:
# Isolate the day in the datetime objects
days_list = []
for date in primary_contrib['contb_receipt_dt'].dt.date:
    days_list.append(date.day)
print(f"There are {len(days_list)} days in this list. Matches our subset dataset's shape.")

# Just to peak at the first 5 days in the list
days_list[:5]

There are 368134 days in this list. Matches our subset dataset's shape.


[5, 6, 4, 5, 6]

In [33]:
# Confirm the amount of days are correct and get value counts of the days
print("Confirmed there are no more than 31 days:", len(pd.Series(days_list).value_counts()))

# value counts automatically sorts our variable, so max count is at the top
days_count = pd.Series(days_list).value_counts().reset_index().rename(columns={"index": "day_of_month"})
print("The day with the most counts is:", days_count[["day_of_month"]].iloc[0].iloc[0])
print("With a count of:", days_count[["count"]].iloc[0].iloc[0])
days_count.head()

Confirmed there are no more than 31 days: 31
The day with the most counts is: 29
With a count of: 21396


,day_of_month,count
0,29,21396
1,31,18997
2,30,18976
3,14,16440
4,9,14785


- **3b answer here:** Most people donate on the 29th day of the month. Our dataset shows this day has 21396 total
    contributions for our candidate.